In [20]:
from funcoesTermosol import *
import pandas as pd
import numpy as np

In [21]:
# Funções para o cálculo da matriz de rigidez

def gauss_seidel(ite, tol, k, f):
    u = np.zeros((k.shape[0], 1))
    u_ant = np.zeros((k.shape[0], 1))
    for n in range(ite):
        for i in range(k.shape[0]): #percorre as linhas da matriz
            soma = 0
            for j in range(k.shape[1]): #percorre as colunas da matriz
                if i != j:
                    soma += k[i, j] * u[j]
            u[i] = (f[i] - soma) / k[i, i]
        erro = ((u - u_ant) / u).max(axis=0)
        if erro < tol:
            return u, n
        u_ant = np.copy(u)
    return u, n

def jacobi(ite, tol, k, f):
    u = np.zeros((k.shape[0], 1))
    u_ant = np.zeros((k.shape[0], 1))
    for n in range(ite):
        for i in range(k.shape[0]): #percorre as linhas da matriz
            soma = 0
            for j in range(k.shape[1]): #percorre as colunas da matriz
                if i != j:
                    soma += k[i, j] * u_ant[j]
            u[i] = (f[i] - soma) / k[i, i]
        erro = ((u - u_ant) / u).max(axis=0)
        if erro < tol:
            return u, n
        u_ant = np.copy(u)
    return u, n

In [22]:
[nn,N,nm,Inc,nc,F,nr,R] = importa('entrada.xls')
dados = [nn,N,nm,Inc,nc,F,nr,R] 
dados

[3,
 array([[0. , 0. , 0.3],
        [0. , 0.4, 0.4]]),
 3,
 array([[1.0e+00, 2.0e+00, 2.1e+11, 2.0e-04],
        [2.0e+00, 3.0e+00, 2.1e+11, 2.0e-04],
        [3.0e+00, 1.0e+00, 2.1e+11, 2.0e-04]]),
 2,
 array([[   0.],
        [   0.],
        [   0.],
        [   0.],
        [ 150.],
        [-100.]]),
 3,
 array([[0.],
        [2.],
        [3.]])]

In [46]:
# Criação da matriz de rigidez global
Kg = np.zeros((2*nn,2*nn))

# Vetor de Comprimentos dos Membros
L_vet = np.zeros((nm,1))

# Vetor de Cossenos dos Membros
Cos_vet = np.zeros((nm,1))

# Vetor de Senos dos Membros
Sen_vet = np.zeros((nm,1))

for i in range(nm):
    n1 = int(Inc[i][0])
    n2 = int(Inc[i][1])
    x_n1 = N[0][n1-1]
    y_n1 = N[1][n1-1]

    x_n2 = N[0][n2-1]
    y_n2 = N[1][n2-1]

    L = np.sqrt((x_n2-x_n1)**2 + (y_n2-y_n1)**2)

    # Definição de vetores importantes
    L_vet[i] = L

    # Cálculo de cosseno e seno
    c = (x_n2-x_n1)/L
    s = (y_n2-y_n1)/L

    # Definição de vetores importantes
    Cos_vet[i] = c
    Sen_vet[i] = s

    E = Inc[i][2]
    A = Inc[i][3]

    k = (E*A)/L * np.array([[c**2, c*s, -c**2, -c*s],  # matriz de rigidez local
                            [c*s, s**2, -c*s, -s**2],
                            [-c**2, -c*s, c**2, c*s],
                            [-c*s, -s**2, c*s, s**2]])

    k = np.array(k)

    indices = ([2*n1-2, 2*n1-1, 2*n2-2, 2*n2-1])
    print(indices)

    for i in range(4):
        for j in range(4):
            Kg[indices[i], indices[j]] += k[i, j]
    
print(f'\nMatriz de rigidez global (Kg):\n{Kg}')
Kg.shape


[0, 1, 2, 3]
[2, 3, 4, 5]
[4, 5, 0, 1]

Matriz de rigidez global (Kg):
[[ 3.0240e+07  4.0320e+07  0.0000e+00  0.0000e+00 -3.0240e+07 -4.0320e+07]
 [ 4.0320e+07  1.5876e+08  0.0000e+00 -1.0500e+08 -4.0320e+07 -5.3760e+07]
 [ 0.0000e+00  0.0000e+00  1.4000e+08  0.0000e+00 -1.4000e+08  0.0000e+00]
 [ 0.0000e+00 -1.0500e+08  0.0000e+00  1.0500e+08  0.0000e+00  0.0000e+00]
 [-3.0240e+07 -4.0320e+07 -1.4000e+08  0.0000e+00  1.7024e+08  4.0320e+07]
 [-4.0320e+07 -5.3760e+07  0.0000e+00  0.0000e+00  4.0320e+07  5.3760e+07]]


(6, 6)

In [24]:
# Remover linhas dos graus de liberdade restritos
R = [int(i[0]) for i in R.tolist()]
Kg = np.delete(Kg, R, axis=0)
F = np.delete(F, R, axis=0)

# Remover colunas dos graus de liberdade restritos
Kg = np.delete(Kg, R, axis=1)

Kg, F, R

(array([[ 1.5876e+08, -4.0320e+07, -5.3760e+07],
        [-4.0320e+07,  1.7024e+08,  4.0320e+07],
        [-5.3760e+07,  4.0320e+07,  5.3760e+07]]),
 array([[   0.],
        [ 150.],
        [-100.]]),
 [0, 2, 3])

In [25]:
# Resolver o sistema de equações para obter os deslocamentos nodais
ite = 100
tol = 1e-10
u_j, n_j = jacobi(ite, tol, Kg, F)
u_gs, n_gs = gauss_seidel(ite, tol, Kg, F)

print("Método de Jacobi")
print(u_j, n_j)
print("-----------------------------")
print("Método de Gauss-Seidel")
print(u_gs, n_gs)

Método de Jacobi
[[-9.52380958e-07]
 [ 1.60714286e-06]
 [-4.01785713e-06]] 99
-----------------------------
Método de Gauss-Seidel
[[-9.52380952e-07]
 [ 1.60714286e-06]
 [-4.01785714e-06]] 22


C:\Users\lucam\AppData\Local\Temp\ipykernel_23184\3022799297.py:29: RuntimeWarning: invalid value encountered in divide
  erro = ((u - u_ant) / u).max(axis=0)
C:\Users\lucam\AppData\Local\Temp\ipykernel_23184\3022799297.py:13: RuntimeWarning: invalid value encountered in divide
  erro = ((u - u_ant) / u).max(axis=0)


In [42]:
# Construção da matriz de deslocamentos nodais
u_desl = np.zeros((2*nn, 1))

c = 0
for i in range(2*nn):
    if i not in R:
        # Condição para extrair os dados da solução de Gauss-Seidel de forma geral
        u_desl[i] = u_gs[c]
        c += 1
    else:
        u_desl[i] = 0

print(f"Deslocamentos [m]:\n{u_desl}")

Deslocamentos [m]:
[[ 0.00000000e+00]
 [-9.52380952e-07]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 1.60714286e-06]
 [-4.01785714e-06]]


In [53]:
# Calculo das reações de apoio	

# Kg * u = Reações de apoio (multiplicação de matrizes) 6x6 * 6x1 = 6x1

# Construção da matriz de reações de apoio
R_apoio = np.zeros((2*nn, 1))

# Realiza a multiplicação de matrizes para obter as reações de apoio 
R_apoio = np.dot(Kg, u_desl)

print(f" Veotr de reações de apoio:\n{R_apoio}")

# As reações de apoio desejadas dependem do vetor de restricoes R
R_apoio_vet = np.zeros((len(R), 1))
c = 0
for r in R:
    R_apoio_vet[c] = R_apoio[r]
    c += 1
    
print(f"\n Reações de apoio [N]: \n{R_apoio_vet}")

 Veotr de reações de apoio:
[[ 7.50000000e+01]
 [ 4.50606308e-09]
 [-2.25000000e+02]
 [ 1.00000000e+02]
 [ 1.50000000e+02]
 [-1.00000000e+02]]

 Reações de apoio [N]: 
[[  75.        ]
 [-224.99999999]
 [ 100.        ]]


In [54]:
# Calculo das Deformações e Tensões nos Elementos

# Inicializa a matriz de deformações
Def_vet = np.zeros((nm,1))

# Inicializa a matriz de tensões
Ten_vet = np.zeros((nm,1))

# Inicializa a matriz de forças
F_vet = np.zeros((nm,1))

# Extração dos módulos de elasticidade e áreas dos elementos
for i in range(nm):
    # Nós do elemento
    n1 = int(Inc[i][0])
    n2 = int(Inc[i][1])

    # Módulo de elasticidade
    E = Inc[i][2]
    L = L_vet[i]

    indices = ([2*n1-2, 2*n1-1, 2*n2-2, 2*n2-1])
    #print(indices)

    # Matriz de deformação do elemento
    vetor_cos_sen = np.transpose(np.array([-Cos_vet[i], -Sen_vet[i], Cos_vet[i], Sen_vet[i]]))
    Def = (1/L) * np.dot(vetor_cos_sen, u_desl[indices])
    Def_vet[i] = Def

    # Matriz de tensão do elemento
    Ten_vet[i] = E * Def

    # Tensão = F / A -> F = E * A * Def
    A = Inc[i][3]
    F_vet[i] = E * A * Def

#print(np.transpose(np.array([-Cos_vet[i], -Sen_vet[i], Cos_vet[i], Sen_vet[i]])).shape)
#print(u_desl[indices].shape)
print(f"Deformações[] \n {Def_vet} \n")
print(f"Tensões internas [Pa] \n {Ten_vet} \n")
print(f"Forças internas [N] \n {F_vet} \n")

Deformações[] 
 [[ 2.38095238e-06]
 [ 5.35714286e-06]
 [-2.97619048e-06]] 

Tensões internas [Pa] 
 [[ 499999.99997747]
 [1124999.99997077]
 [-625000.        ]] 

Forças internas [N] 
 [[ 100.        ]
 [ 224.99999999]
 [-125.        ]] 



In [55]:
# Uso da função exporta para gerar o arquivo de saída
geraSaida('saida', R_apoio_vet, u_desl, Def_vet, Ten_vet, F_vet)